#### AWS t2.xlarge
Vendor=GenuineIntel Model=Intel(R) Xeon(R) CPU E5-2686 v4 @ 2.30GHz MHz=2300.234 bogomips=4600.15
ProcessorChips=1 PhyscalCores=4


In [1]:
using Distributed

In [2]:
HTML("""
<head>
	<meta charset="utf-8">
	<title>Show Babu Frik </title>
	<link rel="stylesheet" href="toasty/toasty.css" />
</head>
<script src="https://code.jquery.com/jquery-1.11.2.min.js"></script>
<script src="toasty/jquery.toasty.js"></script>
""")

HTML{String}("<head>\n\t<meta charset=\"utf-8\">\n\t<title>Show Babu Frik </title>\n\t<link rel=\"stylesheet\" href=\"toasty/toasty.css\" />\n</head>\n<script src=\"https://code.jquery.com/jquery-1.11.2.min.js\"></script>\n<script src=\"toasty/jquery.toasty.js\"></script>\n")

In [3]:
%%javascript

The analogue of IPython's `%%javascript ...code...` in Julia can be constructed by first evaluating

```
macro javascript_str(s) display("text/javascript", s); end
```

to define the `javascript"...."` [string macro](http://docs.julialang.org/en/latest/manual/strings/#non-standard-string-literals) in Julia.  Subsequently, you can simply do:

```
javascript"""
...code...
"""
```

to execute the script in an IJulia notebook.


In [4]:
macro javascript_str(s) display("text/javascript", s); end


@javascript_str (macro with 1 method)

In [5]:
macro hurray()
    return :(
        javascript"""
        $(document).ready( function(){
                  $("body").toasty();
                  $("body").toasty('pop');
        });
        """
    )
end

@hurray (macro with 1 method)

In [6]:
workervec = [("montecarlo@worker_1:22",1),
             ("montecarlo@worker_2:22",1),
             ("montecarlo@worker_3:22",1),
             ("montecarlo@worker_4:22",1)]

4-element Array{Tuple{String,Int64},1}:
 ("montecarlo@worker_1:22", 1)
 ("montecarlo@worker_2:22", 1)
 ("montecarlo@worker_3:22", 1)
 ("montecarlo@worker_4:22", 1)

In [7]:
addprocs(workervec; tunnel=true)

4-element Array{Int64,1}:
 2
 3
 4
 5

In [8]:
println("Number procs: $(nprocs())")
println("Number of workers: $(nworkers())")

Number procs: 5
Number of workers: 4


In [12]:
@everywhere function MC_pi(N::Int)
    in_circle = 0
    for i = 1:N
        x, y = 2*rand()-1, 2*rand()-1
        in_circle += x^2 + y^2 <= 1.0
    end
    return in_circle
end

In [10]:
function Distributed_calc(f::Function, N::Int, workers::Int=-1)
    p = nworkers()
    if workers != -1 && workers <=p
        p = workers
    end
    n = convert(Int64,round(N/p))
    
    n_in = @distributed (+) for i=1:p
        f(n)
    end
    return n_in
end
        

Distributed_calc (generic function with 2 methods)

In [16]:
@time println( (4/1_000_000_000)*MC_pi(1_000_000_000))

3.141687832
 11.503941 seconds (27 allocations: 976 bytes)


In [17]:
@time println( (4/5_000_000_000)*Distributed_calc(MC_PI,5_000_000_000,-1) )
@hurray

3.1415759448
 14.425436 seconds (718 allocations: 26.531 KiB)
